In [1]:
import math
import random

Neuron class

In [2]:
class Neuron:
    def __init__(self, num_inputs):
        self.weights = [random.random() for _ in range(num_inputs)]
        self.bias = random.random()
        self.error = None

    # activatie functie om de output van de neuron te berekenen
    def activate(self, inputs):
        total = sum(w * i for w, i in zip(self.weights, inputs))
        x = total + self.bias
        self.output = 1 / (1 + math.exp(-x))
        return self.output
    
    # berekenen van de error van de output neuron
    def output_error(self, target):
        self.error = self.output * (1 - self.output)
        self.error *= (self.output - target)

    # berekenen van de error van de hidden neuron
    def hidden_error(self, next_layer_weights, next_layer_errors):
        self.error = 0
        for weight, error in zip(next_layer_weights, next_layer_errors):
            self.error += weight * error
        self.error *= self.output * (1 - self.output)
   
    # berekenen van de weight gradients van de neuron
    def compute_weight_gradient(self, prev_layer_output):
        weight_gradients = []
        for prev_output in prev_layer_output:
            weight_gradient = prev_output * self.error
            weight_gradients.append(weight_gradient)
        return weight_gradients
    
    # berekenen van de bias gradient van de neuron
    def compute_bias_gradient(self):
        return self.error
    
    # berekenen van de weight delta van de neuron
    def delta(self, learning_rate, delta_j):
        return [learning_rate * self.output * delta_j for _ in range(len(self.weights))]

    # berekenen van de bias delta van de neuron
    def update(self, learning_rate, weight_gradients):
        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * weight_gradients[i]
        self.bias -= learning_rate * self.bias_gradient

    def __str__(self):
        return f"Weights: {self.weights}, Bias: {self.bias}"

NeuronLayer class

In [3]:
class NeuronLayer:
    def __init__(self, num_neurons, num_inputs):
        self.neurons = [Neuron(num_inputs) for _ in range(num_neurons)]

    # activatie functie om de output van de layer te berekenen
    def activate(self, inputs):
        activated_outputs = []
        for neuron in self.neurons:
            activated_outputs.append(neuron.activate(inputs))
        return activated_outputs

    def __str__(self):
        return '\n'.join(str(neuron) for neuron in self.neurons)

NeuronNetwork class

In [4]:
class NeuronNetwork:
    def __init__(self, num_inputs, num_hidden, num_outputs, learning_rate):
        self.hidden_layer = NeuronLayer(num_hidden, num_inputs)
        self.output_layer = NeuronLayer(num_outputs, num_hidden)
        self.learning_rate = learning_rate

    # activatie functie om de output van de network te berekenen
    def activate(self, inputs):
        hidden_outputs = self.hidden_layer.activate(inputs)
        self.hidden_layer_outputs = hidden_outputs
        return self.output_layer.activate(hidden_outputs)

    # backpropagation functie die ze weights en biases update
    def backpropagate(self, inputs, target):
        outputs = self.activate(inputs)

        # berekenen van de error van de output neuron
        for i, neuron in enumerate(self.output_layer.neurons):
            neuron.output_error(target[i])
            neuron.weight_gradients = neuron.compute_weight_gradient(self.hidden_layer_outputs)
            neuron.bias_gradient = neuron.compute_bias_gradient()

        # berekenen van de error van de hidden neuron
        for i, neuron in enumerate(self.hidden_layer.neurons):
            neuron.hidden_error([neuron.weights[i] for neuron in self.output_layer.neurons],[neuron.error for neuron in self.output_layer.neurons])
            neuron.weight_gradients = neuron.compute_weight_gradient(inputs)
            neuron.bias_gradient = neuron.compute_bias_gradient()

        # update van de weights en biases
        for neuron in self.output_layer.neurons:
            neuron.update(self.learning_rate, neuron.weight_gradients)

        # update van de weights en biases
        for neuron in self.hidden_layer.neurons:
            neuron.update(self.learning_rate, neuron.weight_gradients)

    # train functie die uiteindelijk verandwoordelijk is voor het trainen van het netwerk
    def train(self, inputs, targets, max_runs=50000, min_loss=None):
        run = 0
        while run < max_runs:
            total_loss = 0
            for input_data, target in zip(inputs, targets):
                self.inputs = input_data
                self.backpropagate(input_data, target)
                total_loss += self.get_loss(input_data, target)

            run += 1
            average_loss = total_loss / len(inputs)
            
            # line om de loss per run te printen om zo te zien om deze verbeterd en het model dus aan het leren is
            # print(f"Run {run} - Average Loss: {average_loss}")

            if min_loss is not None and average_loss < min_loss:
                print(f"minimul Loss gehaald")
                break

    # berekend de loss van het netwerk
    def get_loss(self, inputs, target):
        outputs = self.activate(inputs)
        loss = 0.5 * sum((output - target) ** 2 for output, target in zip(outputs, target))
        return loss
    
    def accuracy(y_true, y_pred):
        correct = 0
        for t, p in zip(y_true, y_pred):
            if t == p:
                correct += 1
        total = len(y_true)
        return correct / total

AND-poort trainen

In [5]:
AND = NeuronNetwork(2, 2, 1, 0.1)

train_inputs_and = [[0, 0], [0, 1], [1, 0], [1, 1]]
train_targets_and = [[0], [0], [0], [1]]

AND.train(train_inputs_and, train_targets_and)

print("AND-poort:")
for test_input in train_inputs_and:
    prediction = AND.activate(test_input)
    rounded_prediction = [int(round(output)) for output in prediction]
    print(f"input: {test_input}, prediction: {rounded_prediction}")

# goede output zou zijn:
# [0]
# [0]
# [0]
# [1]

AND-poort:
input: [0, 0], prediction: [0]
input: [0, 1], prediction: [0]
input: [1, 0], prediction: [0]
input: [1, 1], prediction: [1]


XOR-poort trainen

In [6]:
XOR = NeuronNetwork(2, 2, 1, 0.1)

train_inputs_xor = [[0, 0], [0, 1], [1, 0], [1, 1]]
train_targets_xor = [[0], [1], [1], [0]]
XOR.train(train_inputs_xor, train_targets_xor)

print("XOR-poort:")
for test_input in train_inputs_xor:
    prediction = XOR.activate(test_input)
    rounded_prediction = [int(round(output)) for output in prediction]
    print(f"Input: {test_input}, Prediction: {rounded_prediction}")


# goede output zou zijn:
# [0]
# [1]
# [1]
# [0]

XOR-poort:
Input: [0, 0], Prediction: [0]
Input: [0, 1], Prediction: [1]
Input: [1, 0], Prediction: [1]
Input: [1, 1], Prediction: [0]


Half adder trainen

In [7]:
half_adder = NeuronNetwork(2, 2, 2, 0.1)

train_inputs_half_adder = [[0, 0], [0, 1], [1, 0], [1, 1]]
train_targets_half_adder = [[0, 0], [1, 0], [1, 0], [0, 1]]

half_adder.train(train_inputs_half_adder, train_targets_half_adder)

print("Half adder:")
for test_input in train_inputs_half_adder:
    prediction = half_adder.activate(test_input)
    rounded_prediction = [int(round(output)) for output in prediction]
    print(f"input: {test_input}, Prrediction: {rounded_prediction}")



# goede output zou zijn:
# [0, 0]
# [1, 0]
# [1, 0]
# [0, 1]

Half adder:
input: [0, 0], Prrediction: [0, 0]
input: [0, 1], Prrediction: [1, 0]
input: [1, 0], Prrediction: [1, 0]
input: [1, 1], Prrediction: [0, 1]


Trainen Iria-dataset

Hier laad ik de Iris dataset in. Ik verdeel de dataset in trainings- en testsets. Daarna roep ik mijn netwerk aan met het aantal invoerkenmerken, verborgen neuronen en uitvoerkenmerken. Ik train mijn netwerk met behulp van de trainingsgegevens. Na het trainen voorspel ik de labels voor de testgegevens en bereken ik de nauwkeurigheid van de voorspellingen, en print ik de accuracy uit.

In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [9]:
iris = load_iris()
X = iris.data
y = iris.target


encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

num_inputs = 4
num_hidden = 5
num_outputs = 3
learning_rate = 0.1
max_runs = 1000

network = NeuronNetwork(num_inputs, num_hidden, num_outputs, learning_rate)
network.train(X_train, y_train, max_runs=max_runs)

predictions = [network.activate(x) for x in X_test]
predictions = [max(enumerate(p), key=lambda x: x[1])[0] for p in predictions]
true_labels = [list(target).index(1) for target in y_test]
acc = NeuronNetwork.accuracy(true_labels, predictions)
print(f"accuracy = {acc}")


C:\Users\hidde\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


accuracy = 0.9666666666666667


Trainen Digit dataset

In deze code laad ik de digit (soms vage) cijfers en verdeel ik deze in trainingssets en testsets. Vervolgens standaardiseerr ik de gegevens om ze geschikt te maken voor training. ik configureer mijn netwerk met het aantal invoerkenmerken, het aantal verborgen neuronen en het aantal klassen als uitvoer. KWa tijd ben ik er denk ik een paar uurtjes mee bezig geweest, dit is misschien wat lang maar ik was ook nog druk wat andere dingen aan het aanpassen en dit heeft dus ook wat tijd gekost.

In [10]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [11]:
digits = load_digits()
X, y = digits.data, digits.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_inputs = X_train.shape[1]
num_hidden = 16
num_outputs = len(set(y_train)) 
learning_rate = 0.4
max_runs = 10

network = NeuronNetwork(num_inputs, num_hidden, num_outputs, learning_rate)

network.train(X_train, [([1 if i == y else 0 for i in range(num_outputs)]) for y in y_train], max_runs=max_runs, min_loss=0.01)
predictions = [network.activate(x) for x in X_test]
predictions = [max(enumerate(p), key=lambda x: x[1])[0] for p in predictions]
acc = NeuronNetwork.accuracy(y_test, predictions)
print("accuracy = ", acc)



accuracy =  0.95
